In [1]:
# data setup imports
import pandas as pd
from datetime import date
import time
import calendar
import matplotlib.pyplot as plt
import numpy as np

file_name = "../Traffic_Violations.csv"
df = pd.read_csv(file_name)

In [2]:
# Data cleaning/setup - general filtering of data

# Let's drop the columns we deem irrelevant!
filtered_cols_df = df.drop(['Driver City', 'Driver State','DL State', 'Arrest Type','Agency', 'SubAgency', 'Location', 'Latitude', 'Longitude', 'HAZMAT', 'Work Zone', 'Geolocation', 'Charge', 'Article',
                           'Accident', 'Personal Injury', 'Property Damage', 'Fatal', 'Alcohol'], axis=1)

# Change all Date Of Stop dates to weekdays. Hopefully I can find patterns regarding when violations occur during the week
filtered_cols_df['Date Of Stop'] = filtered_cols_df['Date Of Stop'].map(lambda a: time.strftime("%A", time.strptime(a, "%m/%d/%Y")))

# Time Of Stop is now modified with just the hour of the day in military time
filtered_cols_df['Time Of Stop'] = filtered_cols_df['Time Of Stop'].map(lambda a: time.strftime("%H", time.strptime(a, "%H:%M:%S")))


In [3]:
# Data cleaning/setup for drunk driving incidents

# Take out only the drunk driving related incidents
text_driving_df = filtered_cols_df[(filtered_cols_df['Description'] == 'DRIVER USING HANDS TO USE HANDHELD TELEPHONE WHILEMOTOR VEHICLE IS IN MOTION')]
text_driving_df = text_driving_df.drop(['Description', 'Model'], axis=1)

print("Shape of dataframe: ", text_driving_df.shape)

text_driving_df.head(3)
# drunk_driving_df['VehicleType'].value_counts()


Shape of dataframe:  (35387, 14)


,Date Of Stop,Time Of Stop,Belts,Commercial License,Commercial Vehicle,State,VehicleType,Year,Make,Color,Violation Type,Contributed To Accident,Race,Gender
11,Monday,16,No,No,No,VA,02 - Automobile,1996.0,HONDA,SILVER,Citation,No,HISPANIC,M
82,Tuesday,18,No,No,No,MD,02 - Automobile,2002.0,HOND,SILVER,Citation,No,BLACK,M
95,Saturday,10,No,No,No,MD,02 - Automobile,2002.0,TOYT,RED,Citation,No,HISPANIC,F


In [4]:
# Cleaning redundancies, renaming values and correcting mispellings for car make

car_make = {'ACCORD':'HONDA', 'ACCURA':'ACURA', 'ACUR':'ACURA', 'ACURA':'ACURA', \
            'ALTIMA':'NISSAN', 'AUDI':'AUDI', 'BENT':'MERCEDES BENZ', \
            'BENTLEY':'BENTLEY', 'BENZE':'MERCEDES BENZ', 'BLW':'BMW', \
            'BMW':'BMW', 'BMX':'BMW', 'BNW':'BMW', 'BUELL':'BUELL', \
            'BUIC':'BUICK', 'BUICK':'BUICK', 'BUIK':'BUICK', 'BWM':'BMW', \
            'CAD':'CADILLAC', 'CADALIC':'CADILLAC', 'CADALLIC':'CADILLAC', \
            'CADI':'CADILLAC', 'CADILAC':'CADILLAC', 'CADILLAC':'CADILLAC', \
            'CADILLAS':'CADILLAC', 'CADILLIAC':'CADILLAC', 'CADL':'CADILLAC', \
            'CADO':'CADILLAC', 'CAIDILAC':'CADILLAC', 'CEHV':'CHEVROLET', \
            'CEHVORLET':'CHEVROLET', 'CEHVY':'CHEVROLET', 'CHEV':'CHEVROLET', \
            'CHEVERLOT':'CHEVROLET', 'CHEVEROLET':'CHEVROLET', 'CHEVEY':'CHEVROLET', \
            'CHEVORLET':'CHEVROLET', 'CHEVOROLET':'CHEVROLET', 'CHEVR':'CHEVROLET', \
            'CHEVRLET':'CHEVROLET', 'CHEVRLETE':'CHEVROLET', 'CHEVROLERT':'CHEVROLET', \
            'CHEVROLET':'CHEVROLET', 'CHEVROLETY':'CHEVROLET', 'CHEVROLEY':'CHEVROLET', \
            'CHEVY':'CHEVROLET', 'CHEVYC':'CHEVROLET', 'CHRY':'CHRYSLER', 'CHRYL':'CHRYSLER', \
            'CHRYLSER':'CHRYSLER', 'CHRYS':'CHRYSLER', 'CHRYSL':'CHRYSLER', 'CHRYSLER':'CHRYSLER', \
            'CHRYSTLER':'CHRYSLER', 'CHYRSLER':'CHRYSLER', 'CHYSLER':'CHRYSLER', 'COGAR':'FORD', \
            'COOPER':'COOPER', 'CRYSLER':'CHRYSLER', 'DAEWOO':'DAEWOO', 'DAEWOOD':'DAEWOO', 'DIAMONDBACK':'DIAMONDBACK', \
            'DODG':'DODGE', 'DODGE':'DODGE', 'DODGVAL2006':'DODGE', 'DOGDE':'DODGE', 'ELANTRA':'HYUNDAI', 'FERR':'FERRARI', \
            'FERRARI':'FERRARI', 'FIAT':'FIAT', 'FORD':'FORD', 'GEO':'CHEVROLET', 'GMC':'GMC', 'HARL':'HARLEY DAVIDSON', \
            'HARLEY':'HARLEY DAVIDSON', 'HARLEY DAVIDSON':'HARLEY DAVIDSON', 'HENSIM':'HENSIM', 'HIND':'HONDA', \
            'HINDA':'HONDA', 'HINO':'HONDA', 'HIONDA':'HONDA', 'HODNA':'HONDA', 'HOMD':'HONDA', 'HOMDA':'HONDA', \
            'HONA':'HONDA', 'HONAD':'HONDA', 'HOND':'HONDA', 'HONDA':'HONDA', 'HONDAI':'HYUNDAI', 'HUANDAI':'HYUNDAI', \
            'HUDSON':'HUDSON', 'HUMM':'HUMMER', 'HUND':'HYUNDAI', 'HUNDAI':'HYUNDAI', 'HUYN':'HYUNDAI', 'HUYUNDAI':'HYUNDAI', \
            'HYAN':'HYUNDAI', 'HYND':'HYUNDAI', 'HYNDAI':'HYUNDAI', 'HYUANDI':'HYUNDAI', 'HYUDAI':'HYUNDAI', \
            'HYUMDAI':'HYUNDAI', 'HYUN':'HYUNDAI', 'HYUNADAI':'HYUNDAI', 'HYUND':'HYUNDAI', 'HYUNDA':'HYUNDAI', \
            'HYUNDAI':'HYUNDAI', 'HYUNDAIR':'HYUNDAI', 'HYUNDAY':'HYUNDAI', 'HYUNDI':'HYUNDAI', 'HYUNDIA':'HYUNDAI', \
            'INF':'INFINITI', 'INFI':'INFINITI', 'INFIITI':'INFINITI', 'INFIMITI':'INFINITI','INFIN':'INFINITI', \
            'INFINIT':'INFINITI', 'INFINITI':'INFINITI', 'INFINITY':'INFINITI', 'INFINTI':'INFINITI', 'INFINTY':'INFINITI', \
            'INFINTI':'INFINITI', 'INFINTY':'INFINITI', 'INFNITY':'INFINITI', 'INIFINITI':'INFINITI', 'INT':'INFINITI', \
            'ISU':'ISUZU', 'ISUZ':'ISUZU', 'ISUZU':'ISUZU', 'ISZU':'ISUZU', 'ISZUZU':'ISUZU', 'IZUZU':'ISUZU', \
            'JAG':'JAGUAR', 'JAGU':'JAGUAR', 'JAQUAR':'JAGUAR', 'JEEF':'JEEP', 'JEEP':'JEEP', 'JOHN DEERE':'JOHN DEERE', \
            'KAWASAKI':'KAWASAKI', 'KAWK':'KAWASAKI', 'KIA':'KIA', 'KUBOTA':'KUBOTA', 'KYMCO':'KYMCO', 'L300':'MITSUBISHI', \
            'LAND':'LAND ROVER', 'LAND ROVER':'LAND ROVER', 'LANDROVER':'LAND ROVER', 'LEX':'LEXUS', 'LEXS':'LEXUS', \
            'LEXU':'LEXUS', 'LEXUS':'LEXUS', 'LEXUZ':'LEXUS', 'LIN':'LINCOLN', 'LINC':'LINCOLN', 'LINCLN':'LINCOLN', \
            'LINCOLN':'LINCOLN', 'LUXUS':'LEXUS', 'MADA':'MAZDA', 'MADZA':'MAZDA', 'MASERSTI':'MASERATI', 'MAZ':'MAZDA', \
            'MAZA':'MAZDA', 'MAZADA':'MAZDA', 'MAZD':'MAZDA', 'MAZD SW':'MAZDA', 'MAZDA':'MAZDA', 'MECURY':'FORD', \
            'MER':'FORD', 'MERCDES':'MERCEDES BENZ', 'MERCEDES BENZ':'MERCEDES BENZ', 'MERCEDES-BENZ':'MERCEDES BENZ', \
            'MERCEDESE':'MERCEDES BENZ', 'MERCEDEZ':'MERCEDES BENZ', 'MERCEDEZ BENZ':'MERCEDES BENZ', \
            'MERCEDES-BENZ':'MERCEDES BENZ','MERCEDESE':'MERCEDES BENZ','MERCEDEZ':'MERCEDES BENZ', \
            'MERCEDEZ BENZ':'MERCEDES BENZ','MERCEDEZ-BENZ':'MERCEDES BENZ','MERCEDS':'MERCEDES BENZ','MERCURY':'FORD',\
            'MERECEDES':'MERCEDES BENZ','MERZ':'MERZ', 'MERZ B':'MERZ','MERZEDES':'MERCEDES BENZ', 'MINI':'MINI', \
            'MINI COOPER':'MINI','MITIS':'MITSUBISHI','MITISIBISHI':'MITSUBISHI','MITISUBISHI':'MITSUBISHI', \
            'MITS':'MITSUBISHI','MITSABUSHI':'MITSUBISHI', 'MITSBISHI':'MITSUBISHI','MITSIBISH':'MITSUBISHI', \
            'MITSIBISHI':'MITSUBISHI','MITSU':'MITSUBISHI','MITSUBIHI':'MITSUBISHI','MITSUBISH':'MITSUBISHI', \
            'MITSUBISHI':'MITSUBISHI','MITSUBISHUI':'MITSUBISHI','MITTSUBISHI':'MITSUBISHI','MITUBISHI':'MITSUBISHI', \
            'MITZ':'MITSUBISHI','MNNI':'MINI','MONGOOSE':'MONGOOSE','MUSTANG':'FORD','NISS':'NISSAN','NISSA':'NISSAN', \
            'NISSAB':'NISSAN','NISSAM':'NISSAN','NISSAN':'NISSAN','NISSIAN':'NISSAN','NISSON':'NISSAN', 'NISSSN':'NISSAN', \
            'OLDS':'OLDSMOBILE','OLDSMOBILE':'OLDSMOBILE','PLYM':'PLYMOUTH','PLYMOTH':'PLYMOUTH','PLYMOUTH':'PLYMOUTH' ,\
            'PNT':'PONTIAC','PONT':'PONTIAC','PONTAIC':'PONTIAC','PONTIAC':'PONTIAC','PORCHE':'PORSCHE','PORCSCHE':'PORSCHE',\
            'PORS':'PORSCHE','PORSCHE':'PORSCHE','RAM':'RAM','RANG':'LAND ROVER','RANGE':'LAND ROVER','RANGE ROVER':'LAND ROVER',\
            'RAPTOR':'FORD','ROVER':'LAND ROVER','SAA':'SAAB','SAAB':'SAAB','SABB':'SAAB','SAT':'SATURN','SATR':'SATURN', \
            'SATU':'SATURN','SATURN':'SATURN','SCI':'SCION','SCIO':'SCION','SCION':'SCION','SCIONIA':'SCION', 'SCOIN':'SCION', \
            'SMART':'MERCEDES BENZ','SMARTCAR':'MERCEDES BENZ','STRN':'SATURN','SU BARU':'SUBARU','SUB':'SUBARU','SUBA':'SUBARU', \
            'SUBARU':'SUBARU','SUBR':'SUBARU','SUBURU':'SUBARU','SUZI':'SUZUKI','SUZU':'SUZUKI','SUZUKI':'SUZUKI','SX4':'SUZUKI', \
            'TAOT':'TAOTAO','TAOTAO':'TAOTAO','TAOTOA50':'TAOTAO','TESLA':'TESLA','TOMO':'HONDA','TOT':'TOYOTA','TOTOTA':'TOYOTA', \
            'TOTY':'TOYOTA','TOTYOTA':'TOYOTA','TOY':'TOYOTA','TOYATA':'TOYOTA','TOYO':'TOYOTA','TOYOA':'TOYOTA','TOYORA':'TOYOTA',\
            'TOYOTA':'TOYOTA','TOYOTA SCION':'TOYOTA','TOYOTAO':'TOYOTA','TOYOYA':'TOYOTA','TOYPTA':'TOYOTA','TOYT':'TOYOTA', \
            'TOYTA':'TOYOTA','TOYTOA':'TOYOTA','TOYTOTA':'TOYOTA','TOYT`':'TOYOTA','TRIUMPH':'TRIUMPH','VESPA':'VESPA', \
            'VOKLS':'VOLKSWAGEN','VOKS':'VOLKSWAGEN','VOLK':'VOLKSWAGEN','VOLKS':'VOLKSWAGEN','VOLKS WAGON':'VOLKSWAGEN', \
            'VOLKSWAGAN':'VOLKSWAGEN','VOLKSWAGEN':'VOLKSWAGEN','VOLKSWAGON':'VOLKSWAGEN','VOLKWAGEN':'VOLKSWAGEN', \
            'VOLTS':'VOLKSWAGEN','VOLTSWAGON':'VOLKSWAGEN','VOLV':'VOLVO','VOLVO':'VOLVO','VOVLO':'VOLVO','VW':'VOLKSWAGEN', \
            'WILDFIRE':'DODGE','WOLKSWAGEN':'VOLKSWAGEN','WV':'VOLKSWAGEN','YAMAHA':'YAMAHA'}

print("initial shape", text_driving_df.shape)


text_driving_df['Make'] = text_driving_df['Make'].map(car_make)
print(len(text_driving_df['Make'].value_counts()))

text_driving_df['Make'].value_counts(dropna=False)

text_driving_df = text_driving_df.dropna()
print("texting and driving dropped nan ", text_driving_df.shape)
text_driving_df.head(3)

initial shape (35387, 14)
46
texting and driving dropped nan  (33986, 14)


,Date Of Stop,Time Of Stop,Belts,Commercial License,Commercial Vehicle,State,VehicleType,Year,Make,Color,Violation Type,Contributed To Accident,Race,Gender
11,Monday,16,No,No,No,VA,02 - Automobile,1996.0,HONDA,SILVER,Citation,No,HISPANIC,M
82,Tuesday,18,No,No,No,MD,02 - Automobile,2002.0,HONDA,SILVER,Citation,No,BLACK,M
95,Saturday,10,No,No,No,MD,02 - Automobile,2002.0,TOYOTA,RED,Citation,No,HISPANIC,F


In [5]:
# Create dictionaries for mapping to integer values
# Convert data in dataframes to numbers via mapping function

text_driving_df['Time Of Stop'] = text_driving_df['Time Of Stop'].astype(int)
text_driving_df['Year'] = text_driving_df['Year'].astype(float)

weekdays = {k: v for v, k in enumerate(text_driving_df['Date Of Stop'].sort_values().unique())}
print("weekdays dict \n", weekdays)
print()
text_driving_df['Date Of Stop'] = text_driving_df['Date Of Stop'].map(weekdays)

states = {k: v for v, k in enumerate(text_driving_df['State'].sort_values().unique())}
print("State dict \n", states)
print()
text_driving_df['State'] = text_driving_df['State'].map(states)

vehicle_types = {k: v for v, k in enumerate(text_driving_df['VehicleType'].sort_values().unique())}
print("VehicleType dict \n", vehicle_types)
print()
text_driving_df['VehicleType'] = text_driving_df['VehicleType'].map(vehicle_types)

makes = {k: v for v, k in enumerate(text_driving_df['Make'].sort_values().unique())}
print("Make dict \n", makes)
print()
text_driving_df['Make'] = text_driving_df['Make'].map(makes)


colors = {k: v for v, k in enumerate(text_driving_df['Color'].sort_values().unique())}
print("Color dict \n", colors)
print()
text_driving_df['Color'] = text_driving_df['Color'].map(colors)


violations = {k: v for v, k in enumerate(text_driving_df['Violation Type'].sort_values().unique())}
print("violations dict \n", violations)
print()
text_driving_df['Violation Type'] = text_driving_df['Violation Type'].map(violations)


races = {k: v for v, k in enumerate(text_driving_df['Race'].sort_values().unique())}
print("races dict \n", races)
print()
text_driving_df['Race'] = text_driving_df['Race'].map(races)


gender = {k: v for v, k in enumerate(text_driving_df['Gender'].sort_values().unique())}
print("gender dict \n", gender)
print()
text_driving_df['Gender'] = text_driving_df['Gender'].map(gender)


yes_no = {'Yes': 1, 'No': 0}
text_driving_df['Contributed To Accident'] = text_driving_df['Contributed To Accident'].map(yes_no)
# text_driving_df['Accident'] = text_driving_df['Accident'].map(yes_no)
text_driving_df['Belts'] = text_driving_df['Belts'].map(yes_no)
# text_driving_df['Personal Injury'] = text_driving_df['Personal Injury'].map(yes_no)
# text_driving_df['Property Damage'] = text_driving_df['Property Damage'].map(yes_no)
# text_driving_df['Fatal'] = text_driving_df['Fatal'].map(yes_no)
text_driving_df['Commercial License'] = text_driving_df['Commercial License'].map(yes_no)
text_driving_df['Commercial Vehicle'] = text_driving_df['Commercial Vehicle'].map(yes_no)
# text_driving_df['Alcohol'] = text_driving_df['Alcohol'].map(yes_no)

print("Shape before dropping NAN:", text_driving_df.shape)

text_driving_df = text_driving_df.dropna()

print("Shape after dropping NAN:", text_driving_df.shape)

text_driving_df.head(3)

weekdays dict 
 {'Friday': 0, 'Monday': 1, 'Saturday': 2, 'Sunday': 3, 'Thursday': 4, 'Tuesday': 5, 'Wednesday': 6}

State dict 
 {'AK': 0, 'AL': 1, 'AR': 2, 'AZ': 3, 'BC': 4, 'CA': 5, 'CO': 6, 'CT': 7, 'DC': 8, 'DE': 9, 'FL': 10, 'GA': 11, 'IA': 12, 'ID': 13, 'IL': 14, 'IN': 15, 'KS': 16, 'KY': 17, 'LA': 18, 'MA': 19, 'MB': 20, 'MD': 21, 'ME': 22, 'MI': 23, 'MN': 24, 'MO': 25, 'MS': 26, 'NC': 27, 'ND': 28, 'NE': 29, 'NH': 30, 'NJ': 31, 'NM': 32, 'NV': 33, 'NY': 34, 'OH': 35, 'OK': 36, 'ON': 37, 'OR': 38, 'PA': 39, 'QC': 40, 'RI': 41, 'SC': 42, 'TN': 43, 'TX': 44, 'US': 45, 'UT': 46, 'VA': 47, 'VT': 48, 'WA': 49, 'WI': 50, 'WV': 51, 'WY': 52, 'XX': 53}

VehicleType dict 
 {'01 - Motorcycle': 0, '02 - Automobile': 1, '03 - Station Wagon': 2, '04 - Limousine': 3, '05 - Light Duty Truck': 4, '06 - Heavy Duty Truck': 5, '07 - Truck/Road Tractor': 6, '08 - Recreational Vehicle': 7, '09 - Farm Vehicle': 8, '10 - Transit Bus': 9, '12 - School Bus': 10, '19 - Moped': 11, '20 - Commercial Rig':

,Date Of Stop,Time Of Stop,Belts,Commercial License,Commercial Vehicle,State,VehicleType,Year,Make,Color,Violation Type,Contributed To Accident,Race,Gender
11,1,16,0,0,0,47,1,1996.0,15,22,0,0,2,1
82,5,18,0,0,0,21,1,2002.0,15,22,0,0,1,1
95,2,10,0,0,0,21,1,2002.0,43,21,0,0,2,0


In [6]:
# Export cleaned drunk driving data to csv
export_csv = text_driving_df.to_csv (r'..\texting_driving_violations.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path